# Summary

This note book:
- Loads data
- Transforms data to the format compatible with mean comparison tests
- Conducts mean comparison tests (Levene's test, Welch's test, ANOVA test, Tukey's and Games-Howell pairwise comparison)
- Contains a function to stratified-sample one dataframe based on the compositions of another dataframe

# Data Management

In [1]:
## Load the raw data
import pandas as pd
data = pd.read_csv('ab_testing_data.csv')
data.head()

,sku,soid,soname,clid,clname,mkcname,suid,treated,addedtocart,placedorder,trafficcount,after,videocount,grs1month,grs2month,grs3month,grs12month,weightedavgscore,percentilerank,expectedgrs
0,8201MAAA,3,UK,21,Cribs,Nursery,1,0,0,0,14,1,0,0.000000,0.00000,0.000000,0.000000,NaN,NaN,NaN
1,7801KBAA,4,DE,18,Mattress Toppers and Pads,Mattresses - Utility Bedding,7080,0,45,5,1362,1,0,265.558272,857.66493,3759.433838,6547.578930,NaN,NaN,NaN
2,0001ECAA,6,CA,15,Area Rugs,Rugs,1,0,0,0,0,1,0,0.000000,0.00000,0.000000,0.000000,NaN,NaN,NaN
3,3001FCAA,3,UK,15,Area Rugs,Rugs,4312,0,0,0,25,1,0,0.000000,0.00000,0.000000,342.437186,NaN,NaN,NaN
4,5001FCAA,3,UK,15,Area Rugs,Rugs,1,0,1,0,7,1,0,0.000000,0.00000,0.000000,0.000000,NaN,NaN,NaN


In [4]:
data[['sku','suid','treated','after','trafficcount','addedtocart','placedorder']].tail(50)

,sku,suid,treated,after,trafficcount,addedtocart,placedorder
9358804,0141TMSZ,5,0,0,4310,215,26
9358805,0641TMSZ,3418,0,1,418,11,0
9358806,8641TMSZ,1,0,1,13,0,0
9358807,9741TMSZ,5,0,0,30,0,0
9358808,7841TMSZ,3418,0,0,42,0,0
9358809,3001CNSZ,1,0,1,206,10,1
9358810,9601QNSZ,1,0,1,6,0,0
9358811,2611QNSZ,1,0,1,3,0,0
9358812,9811QNSZ,1,0,0,6,0,0
9358813,2021QNSZ,1233,0,1,2,0,0


## Aggregate to SKU Level

In [2]:
aggr = data.groupby(['sku','treated','after']).agg({'trafficcount':'sum', 'addedtocart':'sum', 'placedorder':'sum'}).reset_index()
aggr

,sku,treated,after,trafficcount,addedtocart,placedorder
0,000001AP,0,0,67,3,0
1,000001AP,0,1,43,2,0
2,000001BC,1,0,57,2,0
3,000001BC,1,1,66,2,0
4,000001BQ,0,0,396,6,0
...,...,...,...,...,...,...
3806670,9999YCUE,0,1,0,0,0
3806671,9999YHV,0,0,8,0,0
3806672,9999YHV,0,1,15,2,2
3806673,9999ZHAD,0,0,0,0,0


In [21]:
# Save works to csv
aggr.to_csv('aggregated_ab_testing_data.csv', index = False)

In [1]:
import pandas as pd
aggr = pd.read_csv('aggregated_ab_testing_data.csv')

## Differencing between before and after

In [24]:
## Pivot the table so that we have a before and an after column for each metric
pv = aggr.pivot('sku','after').dropna() 
# also drops NaN to eliminate SKUs that did not have full before-after conditions
print('-----Pivot Table-----')
print(pv)

## Subtract before from after for each metric
difTraffic = pv['trafficcount',1] - pv['trafficcount',0]
difCart = pv['addedtocart',1] - pv['addedtocart',0]
difBuy = pv['placedorder',1] - pv['placedorder',0]

## Make dataframe from the differences 
df = pd.DataFrame({'treated':pv['treated',0], 'difTraffic':difTraffic, 'difCart':difCart, 'difBuy': difBuy}).reset_index()
    ### reset_index() passes the current index, which is sku, as a column.
    ### reset_index() is optional, but I prefer a manipulable column and an iterable index in most cases
print('')
print('-----Difference Table-----')
df

-----Pivot Table-----
         treated      trafficcount        addedtocart      placedorder     
after          0    1            0      1           0    1           0    1
sku                                                                        
000001AP     0.0  0.0         67.0   43.0         3.0  2.0         0.0  0.0
000001BC     1.0  1.0         57.0   66.0         2.0  2.0         0.0  0.0
000001BQ     0.0  0.0        396.0  317.0         6.0  2.0         0.0  0.0
000001CO     0.0  0.0         92.0   14.0         3.0  0.0         0.0  0.0
000001DG     0.0  0.0          3.0    3.0         0.0  0.0         0.0  0.0
...          ...  ...          ...    ...         ...  ...         ...  ...
9999XSGB     0.0  0.0          0.0    3.0         0.0  0.0         0.0  0.0
9999XWCN     0.0  0.0          9.0    7.0         1.0  1.0         0.0  0.0
9999YADV     0.0  0.0         14.0   41.0         0.0  0.0         0.0  0.0
9999YHV      0.0  0.0          8.0   15.0         0.0  2.0        

,sku,treated,difTraffic,difCart,difBuy
0,000001AP,0.0,-24.0,-1.0,0.0
1,000001BC,1.0,9.0,0.0,0.0
2,000001BQ,0.0,-79.0,-4.0,0.0
3,000001CO,0.0,-78.0,-3.0,0.0
4,000001DG,0.0,0.0,0.0,0.0
...,...,...,...,...,...
1864152,9999XSGB,0.0,3.0,0.0,0.0
1864153,9999XWCN,0.0,-2.0,0.0,0.0
1864154,9999YADV,0.0,27.0,0.0,0.0
1864155,9999YHV,0.0,7.0,2.0,2.0


In [44]:
## Save works to csv
df.to_csv('differenced_data.csv', index = False)

In [2]:
df = pd.read_csv('differenced_data.csv')

# Statistical Analyses - Mean Comparison

## Levene's Test for Equal Variance

First, we must test whether the variance in differenced values are "equal" between two groups, because the result would then decide which mean comparison test to use: ANOVA or Welch's test. We can use Levene's Test for Variance Equality.

pingouin allows passing a value column and a group (categorical) column straight from the dataframe. Other packages might require passing multiple series, one series for each group.

In [25]:
import pingouin as pg
def levene_test(data, value_field, category_field):
    return pg.homoscedasticity(data, dv=value_field, group=category_field, method='levene', alpha=0.05)

In [26]:
print('Variance Equality between control and treatment in Traffic difference')
print(levene_test(df, 'difTraffic', 'treated'))

print('----------')
print('Variance Equality in Added-to-Cart Clicks difference')
print(levene_test(df, 'difCart', 'treated'))

print('----------')
print('Variance Equality in Purchases difference')
print(levene_test(df, 'difBuy', 'treated'))

Variance Equality between control and treatment in Traffic difference
                  W  pval  equal_var
levene  1819.622559   0.0      False
----------
Variance Equality in Added-to-Cart Clicks difference
                  W  pval  equal_var
levene  1929.142058   0.0      False
----------
Variance Equality in Purchases difference
                  W  pval  equal_var
levene  1530.320935   0.0      False


The control and treatment groups differ in variances of all metrics' differences. We shall use:
- Welch's test instead of ANOVA for mean differences 
- Games-Howell test for post-hoc pairwise comparison instead of Tukey's test.

## Test of Difference in Means: ANOVA vs. Welch's Test

Both ANOVA and Welch's Test are used to compare the difference in means across multiple groups.

### ANOVA

When the variances are equal between groups, we can use ANOVA.

In [27]:
print('ANOVA on Traffic Difference')
print(pg.anova(dv='difTraffic', between='treated', data=df))

print('----------')
print('ANOVA on Added-to-Cart Difference')
print(pg.anova(dv='difCart', between='treated', data=df))

print('----------')
print('ANOVA on Purchase Difference')
print(pg.anova(dv='difBuy', between='treated', data=df))

ANOVA on Traffic Difference
    Source  ddof1    ddof2         F     p-unc       np2
0  treated      1  1864155  1.874216  0.170993  0.000001
----------
ANOVA on Added-to-Cart Difference
    Source  ddof1    ddof2           F         p-unc     np2
0  treated      1  1864155  186.352751  1.997701e-42  0.0001
----------
ANOVA on Purchase Difference
    Source  ddof1    ddof2           F          p-unc       np2
0  treated      1  1864155  722.199889  4.772576e-159  0.000387


### Welch's Test

When the variances are unequal between groups, we use Welch's Test. This is the appropriate test for our case.

In [28]:
from pingouin import welch_anova

print("Welch's test on Traffic Difference")
print(welch_anova(dv='difTraffic', between='treated', data=df))

print('----------')
print("Welch's test on Added-to-Cart Difference")
print(welch_anova(dv='difCart', between='treated', data=df))

print('----------')
print("Welch's test on Purchase Difference")
print(welch_anova(dv='difBuy', between='treated', data=df))

Welch's test on Traffic Difference
    Source  ddof1          ddof2         F     p-unc       np2
0  treated      1  199886.508265  0.564182  0.452581  0.000001
----------
Welch's test on Added-to-Cart Difference
    Source  ddof1          ddof2          F         p-unc     np2
0  treated      1  196227.780707  41.469138  1.200217e-10  0.0001
----------
Welch's test on Purchase Difference
    Source  ddof1         ddof2           F         p-unc       np2
0  treated      1  194135.64732  127.607125  1.397569e-29  0.000387


## Post-hoc Pairwise Comparison: Tukey's vs Games-Howell

After ANOVA or Welch's test, we should compare the means between the groups. This step is not necessary when comparing between just two groups, because we already know their means, variances, and whether the difference between them is significant. However, when we have more than two groups to compare, doing a t-test, ANOVA, or Welch' test for each pair in a large collection of groups would accumulate the errors of these tests. It is thus important to use pairwise comparison to examine the differences between each pair without widening error. Which pairwise test to use also depends on equal or unequal variances.

### Tukey's Test

For equal variances among groups, we can use Tukey's test. 

In [29]:
def tukey(data, value_field, category_field, n_round):
    return pg.pairwise_tukey(data=data, dv=value_field,
                        between=category_field).round(n_round)

In [30]:
print("Tukey's Pairwise Comparison on Traffic Difference")
print(tukey(df, 'difTraffic', 'treated', 1))

print('----------')
print("Tukey's on Added-to-Cart Difference")
print(tukey(df, 'difCart', 'treated', 1))

print('----------')
print("Tukey's on Purchase Difference")
print(tukey(df, 'difBuy', 'treated', 1))

Tukey's Pairwise Comparison on Traffic Difference
     A    B  mean(A)  mean(B)  diff   se    T  p-tukey  hedges
0  0.0  1.0     21.7     18.1   3.6  2.6  1.4      0.2     0.0
----------
Tukey's on Added-to-Cart Difference
     A    B  mean(A)  mean(B)  diff   se     T  p-tukey  hedges
0  0.0  1.0      0.2     -2.8   3.0  0.2  13.7      0.0     0.0
----------
Tukey's on Purchase Difference
     A    B  mean(A)  mean(B)  diff   se     T  p-tukey  hedges
0  0.0  1.0     -0.3     -1.9   1.6  0.1  26.9      0.0     0.1


### Games-Howell Test

For unequal variances among groups, we can use Games-Howell post-hoc pairwise comparison. This is the appropriate test for our case. 

In [31]:
def games_howell(data, value_field, category_field, n_round):
    return pg.pairwise_gameshowell(data=data, dv=value_field,
                        between=category_field).round(n_round)

In [32]:
print("Games-Howell Pairwise Comparison on Traffic Difference")
print(games_howell(df, 'difTraffic', 'treated', 1))

print('----------')
print("Games-Howell on Added-to-Cart Difference")
print(games_howell(df, 'difCart', 'treated', 1))

print('----------')
print("Games-Howell on Purchase Difference")
print(games_howell(df, 'difBuy', 'treated', 1))

Games-Howell Pairwise Comparison on Traffic Difference
     A    B  mean(A)  mean(B)  diff   se    T        df  pval  hedges
0  0.0  1.0     21.7     18.1   3.6  4.8  0.8  199886.5   0.5     0.0
----------
Games-Howell on Added-to-Cart Difference
     A    B  mean(A)  mean(B)  diff   se    T        df  pval  hedges
0  0.0  1.0      0.2     -2.8   3.0  0.5  6.4  196227.8   0.0     0.0
----------
Games-Howell on Purchase Difference
     A    B  mean(A)  mean(B)  diff   se     T        df  pval  hedges
0  0.0  1.0     -0.3     -1.9   1.6  0.1  11.3  194135.6   0.0     0.0


# Appendix: Stratified-Sampling of Control group based on Treated group

One might be concerned that the difference in product category compositions between the control group and treated group could translate to inherent different behaviors that cannot be explained by the treatment. To account for this phenomenon, we can randomly select from the control group a sample that has similar composition with that of the treated group.

#### Add categories to df and compare the compositions of treated and control based on categories

In [6]:
# Create a dataframe with unique SKUs and categories
cat = data[['sku','mkcname']].drop_duplicates()
# optionally save it:
# cat.to_csv('macrocategories.csv', index = False)

# Merge df with cat to have a category column
df_cat = df.merge(cat, on = 'sku')
df_cat.head()

,sku,treated,difTraffic,difCart,difBuy,mkcname
0,000001AP,0.0,-24.0,-1.0,0.0,Entertainment Furniture
1,000001BC,1.0,9.0,0.0,0.0,Rugs
2,000001BQ,0.0,-79.0,-4.0,0.0,Entertainment Furniture
3,000001CO,0.0,-78.0,-3.0,0.0,Rugs
4,000001DG,0.0,0.0,0.0,0.0,Outdoor


In [7]:
# Separate control and treated group for easier operations:
control = df_cat[df_cat['treated'] == 0]
treated = df_cat[df_cat['treated'] == 1]
print('control size: ', len(control))
print('treatment size: ', len(treated))

control size:  1672813
treatment size:  191344


In [8]:
# Compare the compositions of treated and control based on categories
## Function to compare composition between treated and control:
def compare_treatment_control(treated, control):
    treated_structure = pd.DataFrame(treated['mkcname'].value_counts(normalize = True)).reset_index().rename(columns = {'index':'macrocategory','mkcname':'treated_pct'})
    control_structure = pd.DataFrame(control['mkcname'].value_counts(normalize = True)).reset_index().rename(columns = {'index':'macrocategory','mkcname':'control_pct'})
    return pd.merge(treated_structure, control_structure, on = 'macrocategory').round(3)

compare_treatment_control(treated, control)

,macrocategory,treated_pct,control_pct
0,Wall Art,0.598,0.593
1,Rugs,0.303,0.217
2,Furniture - Kitchen and Dining,0.020,0.020
3,Furniture - Bedroom,0.010,0.021
4,Outdoor,0.009,0.006
5,Entertainment Furniture,0.009,0.028
6,Upholstery - Core,0.008,0.013
7,Lighting,0.006,0.021
8,Plumbing - Core,0.006,0.005
9,Furniture - Home Office,0.006,0.008


#### Formulate the stratification

For the purpose of generalization, let's call the control group the sampled group, because this is the group to be sampled, and the treated group the base group, because this is the group on which to base the stratification.


Pandas has the "pd.DataFrame.sample(frac = fraction)" method that allows random sampling with a specific fraction from a dataframe. 

The idea is, for the sampled group, for each category we randomly select a tailored fraction and then add the samples from all categories together into a final sample dataframe. These fractions vary from category to category, in such a way that in the end, the number of items in that category divided by the length of the entire sample dataframe should equal the percentage of that category in the base dataframe. 

__Mathematically:__

**_fraction_ * subtotal / sample_size = base_pct**

where:
- fraction is the fraction to sample from a certain category of the sampled group
- subtotal is the number of items within that category in the resulting sample 
- sample_size is the size of the desired sample, or the length of the entire sample dataframe
- base_pct is the percentage of the items within the same category in the base group

Now, for each category we need to get the _fraction_ from which to sample, so:

**_fraction_ = base_pct * grand_total / subtotal**

In [38]:
from tqdm import tqdm
def mirrorCompositionStratifiedSample(sampled_group, base_group, # dataframes of sampled group and base group
                                      category = None, sampled_category = None, base_category = None, # choose category column
                                      sample_size = None, # if sample size is not specified, it will be the base group's size
                                      random_state = None): # optionally set random state for reproducibility
    if category is not None:
        sampled_category = category
        base_category = category
    if sample_size is None:
        sample_size = len(base_group)
        
    # Create a base structure with percentages of all categories
    base_structure = pd.DataFrame(base_group[base_category].value_counts(normalize = True)).reset_index()
    base_structure.rename(columns = {base_structure.columns[0]:base_category, base_structure.columns[1]:'base_pct'}, inplace = True)    

    # Create the sample dataframe
    sample = pd.DataFrame()

    for i in tqdm(range(len(base_structure))):
        # for each category, randomly select from the sampled group in that category only, by a fraction equal to 
        # fraction * subtotal / sample_size = base_pct
        # fraction = base_pct * sample_size / sub total
        base_pct = base_structure['base_pct'][i]
        sub_sample = sampled_group[sampled_group[sampled_category] == str(base_structure[base_category][i])]
        subtotal = len(sub_sample)
        fraction = base_pct * sample_size / subtotal
        # Use the fraction to do randomly sample from this certain category
        sampling = sub_sample.sample(frac = fraction, replace = True, random_state = random_state) 
        ## replace (i.e., sampling one row more than once) must be allowed when
        ## there are fewer items in the sampled group than the base group for the same category
        
        # Union the sample from the category to the main sample dataframe
        sample = pd.concat([sample, sampling])
    return sample

In [43]:
# Create the stratified sample of the control group based on the composition of the treated group
control_sample = mirrorCompositionStratifiedSample(sampled_group = control, base_group = treated, category = "mkcname",
                                                  random_state = 2022)
control_sample

100%|██████████| 26/26 [00:03<00:00,  8.16it/s]


,sku,treated,difTraffic,difCart,difBuy,mkcname
659044,3507ABID,0.0,0.0,0.0,0.0,Wall Art
1481499,7932OBUE,0.0,0.0,0.0,0.0,Wall Art
280032,1481GHAD,0.0,0.0,0.0,0.0,Wall Art
195919,1032ZJW,0.0,12.0,-1.0,0.0,Wall Art
830557,4424CDBJ,0.0,-22.0,-4.0,0.0,Wall Art
...,...,...,...,...,...,...
712701,3801RZBO,0.0,-1.0,-23.0,-5.0,Pet
1606133,8601FTEP,0.0,178.0,-2.0,0.0,Pet
935580,5001LKLK,0.0,70.0,0.0,0.0,Pet
174034,0921DLQJ,0.0,54.0,-22.0,2.0,Pet


In [44]:
# If the column names of the same categorical variable differ between 2 groups, we can specify the category name for each:
sample_example = mirrorCompositionStratifiedSample(sampled_group = control.rename(columns = {'mkcname':'macrocategory'}), 
                                                   base_group = treated.rename(columns = {'mkcname':'macro_category'}), 
                                                   sampled_category = "macrocategory",
                                                   base_category = 'macro_category',
                                                  random_state = 2022)
sample_example

100%|██████████| 26/26 [00:02<00:00,  8.76it/s]


,sku,treated,difTraffic,difCart,difBuy,macrocategory
659044,3507ABID,0.0,0.0,0.0,0.0,Wall Art
1481499,7932OBUE,0.0,0.0,0.0,0.0,Wall Art
280032,1481GHAD,0.0,0.0,0.0,0.0,Wall Art
195919,1032ZJW,0.0,12.0,-1.0,0.0,Wall Art
830557,4424CDBJ,0.0,-22.0,-4.0,0.0,Wall Art
...,...,...,...,...,...,...
712701,3801RZBO,0.0,-1.0,-23.0,-5.0,Pet
1606133,8601FTEP,0.0,178.0,-2.0,0.0,Pet
935580,5001LKLK,0.0,70.0,0.0,0.0,Pet
174034,0921DLQJ,0.0,54.0,-22.0,2.0,Pet


In [37]:
# Compare the treated group and the control sample
print('control sample size: ', len(control_sample))
print('treatment size: ', len(treated))
compare_treatment_control(treated, control_sample)

control sample size:  191344
treatment size:  191344


,macrocategory,treated_pct,control_pct
0,Wall Art,0.598,0.598
1,Rugs,0.303,0.303
2,Furniture - Kitchen and Dining,0.020,0.020
3,Furniture - Bedroom,0.010,0.010
4,Outdoor,0.009,0.009
5,Entertainment Furniture,0.009,0.009
6,Upholstery - Core,0.008,0.008
7,Lighting,0.006,0.006
8,Plumbing - Core,0.006,0.006
9,Furniture - Home Office,0.006,0.006


The size and composition of the control sample now mirrors those of the treated group.

In [34]:
# Re-create df by unioning treated and the stratified sample of the control group
df = pd.concat([treated, control_sample])
df

,sku,treated,difTraffic,difCart,difBuy,mkcname
1,000001BC,1.0,9.0,0.0,0.0,Rugs
13,000001SH,1.0,-123.0,0.0,0.0,Furniture - Bedroom
19,000001ZJ,1.0,23.0,-8.0,-2.0,Rugs
23,00001AMR,1.0,152.0,-22.0,-10.0,Rugs
53,00001EBW,1.0,8.0,-15.0,-4.0,Fireplaces & Grills
...,...,...,...,...,...,...
712701,3801RZBO,0.0,-1.0,-23.0,-5.0,Pet
1606133,8601FTEP,0.0,178.0,-2.0,0.0,Pet
935580,5001LKLK,0.0,70.0,0.0,0.0,Pet
174034,0921DLQJ,0.0,54.0,-22.0,2.0,Pet


In [35]:
# Re-do the statistics on a more balanced dataframe

print("Games-Howell Pairwise Comparison on Traffic Difference")
print(games_howell(df, 'difTraffic', 'treated', 1))

print('----------')
print("Games-Howell on Added-to-Cart Difference")
print(games_howell(df, 'difCart', 'treated', 1))

print('----------')
print("Games-Howell on Purchase Difference")
print(games_howell(df, 'difBuy', 'treated', 1))

Games-Howell Pairwise Comparison on Traffic Difference
     A    B  mean(A)  mean(B)  diff   se    T        df  pval  hedges
0  0.0  1.0     10.1     18.1  -8.1  5.0 -1.6  231834.4   0.1    -0.0
----------
Games-Howell on Added-to-Cart Difference
     A    B  mean(A)  mean(B)  diff   se    T        df  pval  hedges
0  0.0  1.0     -0.1     -2.8   2.6  0.5  5.5  217197.1   0.0     0.0
----------
Games-Howell on Purchase Difference
     A    B  mean(A)  mean(B)  diff   se     T        df  pval  hedges
0  0.0  1.0     -0.2     -1.9   1.7  0.1  11.5  205173.6   0.0     0.0
